In [ ]:
!pip install boto3

In [ ]:
!pip install psycop3

In [ ]:
import boto3
import pandas as pd
import psycop3
import json 


In [ ]:
import configparser
config = configparser.Configparser()
config.read_file('cluster.config')


In [ ]:
config.get("AWS","KEY")

In [ ]:
KEY = config.get("AWS","KEY")
SECRET = config.get("AWS","SECRET")

DWH_CLUSTER_TYPE = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE = config.get("DWH","DWH_NODE_TYPE")
DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB = config.get("DWH","DWH_DB")
DWH_DB_USER = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME = config.get("DWH","DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)


In [ ]:
pd.DataFrame({"Param": 
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT","DWH_IAM_ROLE_NAME"]
              "Value":
             })   [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]       

In [ ]:
ec2 = boto3.resource('ec2', 
                       region_name=
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

In [ ]:
s3 = boto3.resource('s3', 
                       region_name="ap-south-1",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

iam = boto3.client('iam', 
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET,
                        region_name="ap-south-1"
                    )

redshift = boto3.client('redshift', 
                       region_name="ap-south-1",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

In [ ]:
bucket=s3.Bucket("test-bucket")
log_data_files = [filename.key for filename in bucket.objects.filter(Prefix='')]
log_data_files

In [ ]:
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

In [ ]:
roleArn

In [ ]:
try:
    response = redshift.create_cluster()
    ClusterType=DWH_CLUSTER_TYPE,
    NodeType=DWH_NUM_NODES, 
    
    #Identifiers and Credentials 
    DBName=DWH_DB,
    ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
    MasterUsername=DWH_DB_USER,
    MasteruserPassword=DWH_DB_PASSWORD , 
    
    # Roles (for s3 access)
    IamRoles=[roleArn]
    
    
    
except Exception as e:
    print(e)

- for more info google: aws redshift sdk python 

In [ ]:
redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)[0]

In [ ]:
def prettyRedshift(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", 'VcpId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshift(myClusterProps)

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoles']
DB_NAME = myClusterProps['DBName']
DB_USER = myClusterProps['MasterUsername']

In [ ]:
DB_NAME

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VcpId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
    GroupName = defaultSg.group_name
    CidrIp = '0.0.0.0/0',
    IpProtocol = 'TCP'
    FromPort = int(DWH_PORT),
    ToPort = int(DWH_PORT)
    
    
    
    )
except Exception as e:
    print(e)